# Instalação do pydantic e fastapi

In [1]:
!pip install pydantic[email]
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00


# Importações necessárias

In [12]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI() #Criação do app FastAPI

In [3]:
''' Classe que representa os dados de determinado paciente de forma estruturada
usando o basemodel do pydantic'''
class Patient(BaseModel):

    #guarda o id do paiente como uuid4 data, o field possibilita acrescentar metadados
    #gera um uuid4 automaticamente quando um objeto patient é criado
    id: UUID4 = Field(default_factory=uuid4, description="Unique identifier for the patient")

    #armazena o nome completo de um paciente, que será uma string
    name: str = Field(..., description="Patient's full name")

    #armazena o emai do paciente, que deverá ser um EmailStr para ser válido
    email: EmailStr = Field(..., description="Patient's email")

    #armazena a data de nascimento do paciente como um objeto datetime
    date_of_birth: datetime = Field(..., description="Date of birth")

    #armazena de forma opcional as alergias do paciente, cria uma lista vazia para caso
    # o paciente não tenha alergias
    allergies: Optional[list[str]] = Field(default=[], description="List of allergies")

    #armazena de forma ocpional as condições médicas do paciente, cria uma lista vazia
    # para caso o paciente não tenha condições médicas específicadas
    medical_conditions: Optional[list[str]] = Field(default=[], description="Pre-existing medical conditions")

In [4]:
'''Criação da classe medicalrecords, para representar o registro médico de um
paciente usando o basemodel do pydantic'''
class MedicalRecord(BaseModel):

    #guarda o id do registro médico como uuid4 data, o field possibilita acrescentar metadados
    #gera um uuid4 automaticamente quando um objeto medicalrecord é criado
    id: UUID4 = Field(default_factory=uuid4, description="Unique ID for the record")

    #guarda o id do paciente relacionado a este registro médico
    patient_id: UUID4 = Field(..., description="ID of the patient")

    #armazena a data e hora em que o registro é criado
    date: datetime = Field(default_factory=datetime.now, description="Date of the record")

    #armazena o diagnóstico para o registro
    diagnosis: str = Field(..., description="Medical diagnosis")

    #armazena informações sobre o tratamento prescrito
    treatment: str = Field(..., description="Prescribed treatment")

    #armazena o nomedo médico responsável
    doctor: str = Field(..., description="Name of the doctor")

# Criação de listas para armazenar as informações

In [5]:
#Banco de dados fictício
patients_db = []
medical_records_db = []

# Endpoints da API

In [13]:
# os endpoints ligam as funções criadas aos caminhos corretos dentro da aplicação
# permitindo acessar e manipular os dados

# cria a função create_patient, que recebe um objeto paciente e o adiciona
# no banco de dados fictício criado para armazenar os dados dos pacientes
# e retorna o paciente criado como resposta
@app.post("/patients/", response_model=Patient)
def create_patient(patient: Patient):
    patients_db.append(patient)
    return patient

# cria uma função get_patients que retorna a lista completa de pacientes
@app.get("/patients/", response_model=list[Patient])
def get_patients():
    return patients_db

# cria uma função create_medical_record que checa se o patient id é válido e existe
# no banco de dados dos pacientes, se ele existir cria uma objeto medical_record e
# o adiciona no banco de dados
@app.post("/medical_records/", response_model=MedicalRecord)
def create_medical_record(record: MedicalRecord):
    if not any(p.id == record.patient_id for p in patients_db):
        raise HTTPException(status_code=404, detail="Patient not found")
    medical_records_db.append(record)
    return record

# cria a função get_medical_records que filtra pelo id do paciente os registros
# presentes no banco de dados, encontrando todos os registros daquele paciente
# se forem encontrados registros eles serão retornados como resposta
@app.get("/medical_records/{patient_id}", response_model=list[MedicalRecord])
def get_medical_records(patient_id: UUID4):
    records = [r for r in medical_records_db if r.patient_id == patient_id]
    if not records:
        raise HTTPException(status_code=404, detail="No records found for this patient")
    return records


Criação de um cliente para realizar testes

In [14]:
client = TestClient(app)

# Seção de testes da API utilizando o client de fastapi e validações do pydantic

In [17]:
# teste de criação de um novo objeto de paciente, cujos dados serão armazenados em
# formato json, verificação da resposta utilizando o assert e printando a resposta
# em seguida. Ele cria um paciente chamado Anderson
def test_create_patient():
    response = client.post("/patients/", json={
        "name": "Anderson Soares",
        "email": "andersonsoares@example.com",
        "date_of_birth": "1990-01-01T00:00:00",
        "allergies": ["Peanuts"],
        "medical_conditions": ["Asthma"]
    })
    assert response.status_code == 200
    print("Created Patient:", response.json())
    return response.json()

# teste de criação de um novo registro médico, para testar todas as funcionalidades
# ele também cria um novo paciente, Jair Santos e em seguida cria seu registro médico
# com as informações apresentadas
def test_create_medical_record():
    patient_response = client.post("/patients/", json={
        "name": "Jair Santos",
        "email": "jairsantos@example.com",
        "date_of_birth": "1985-05-05T00:00:00",
        "allergies": [],
        "medical_conditions": []
    })
    patient_id = patient_response.json()["id"]
    print("Created Patient ID:", patient_id)

    record_response = client.post("/medical_records/", json={
        "patient_id": patient_id,
        "diagnosis": "Flu",
        "treatment": "Rest and hydration",
        "doctor": "Dr. Smith"
    })

    assert record_response.status_code == 200
    print("Created Medical Record:", record_response.json())
    return record_response.json()

# Executando os testes e exibindo os resultados
if __name__ == "__main__":
    patient = test_create_patient()
    medical_record = test_create_medical_record()


Created Patient: {'id': '74f7e3d4-5a54-4240-ab8a-5581fe117406', 'name': 'Anderson Soares', 'email': 'andersonsoares@example.com', 'date_of_birth': '1990-01-01T00:00:00', 'allergies': ['Peanuts'], 'medical_conditions': ['Asthma']}
Created Patient ID: 3bb175c1-2803-4ba9-abce-0aea4b669982
Created Medical Record: {'id': '99a46e62-618a-4023-8daa-accffe2bfac2', 'patient_id': '3bb175c1-2803-4ba9-abce-0aea4b669982', 'date': '2025-03-13T12:08:40.345197', 'diagnosis': 'Flu', 'treatment': 'Rest and hydration', 'doctor': 'Dr. Smith'}
